#### Import required modules

In [61]:
import pandas as pd
import matplotlib
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [62]:
# from google.colab import drive
# drive.mount('/content/drive')

In [63]:
df = pd.read_csv(r"C:\Users\TOBENNA\Desktop\FINAL YEAR PROJECT\CRIME IN LA\dataset.csv")
df

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 0:00,03/01/2020 0:00,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 0:00,02/08/2020 0:00,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 0:00,11/04/2020 0:00,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 0:00,03/10/2020 0:00,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932135,241605270,02/11/2024 0:00,02/11/2024 0:00,1920,16,Foothill,1681,2,888,TRESPASSING,...,AA,Adult Arrest,888.0,NaN,NaN,NaN,8700 SHARP AV,NaN,34.2278,-118.4051
932136,241604405,01/14/2024 12:00:00 AM,01/13/2024 12:00:00 AM,720,16,Foothill,1673,1,230,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",...,IC,Invest Cont,230.0,NaN,NaN,NaN,8700 LANKERSHIM BL,NaN,34.2271,-118.3878
932137,242106032,02/20/2024 12:00:00 AM,02/17/2024 12:00:00 AM,2330,21,Topanga,2103,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,IC,Invest Cont,420.0,NaN,NaN,NaN,22700 NAPA ST,NaN,34.2260,-118.6214
932138,242004546,01/16/2024 12:00:00 AM,01/16/2024 12:00:00 AM,1510,20,Olympic,2013,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,HOBART BL,3RD ST,34.0690,-118.3054


In [64]:
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'], format='mixed')
df['Year OCC'] = df['DATE OCC'].dt.year
df['Month OCC'] = df['DATE OCC'].dt.month
df['DAY OCC'] = df['DATE OCC'].dt.day
df['DAY WEEK OCC'] = df['DATE OCC'].dt.day_name()

df.drop(columns=['DATE OCC'], inplace=True)
df.head()

,DR_NO,Date Rptd,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Year OCC,Month OCC,DAY OCC,DAY WEEK OCC
0,190326475,03/01/2020 0:00,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,NaN,...,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,2020,3,1,Sunday
1,200106753,02/09/2020 0:00,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,1822 1402 0344,...,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628,2020,2,8,Saturday
2,200320258,11/11/2020 0:00,1700,3,Southwest,356,1,480,BIKE - STOLEN,0344 1251,...,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002,2020,11,4,Wednesday
3,200907217,05/10/2023 0:00,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,...,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387,2020,3,10,Tuesday
4,220614831,08/18/2022 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,1822 1501 0930 2004,...,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277,2020,8,17,Monday


In [65]:
df.drop(columns=['Date Rptd'], inplace=True)
df.head()

,DR_NO,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,...,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Year OCC,Month OCC,DAY OCC,DAY WEEK OCC
0,190326475,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,NaN,0,...,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,2020,3,1,Sunday
1,200106753,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,1822 1402 0344,47,...,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628,2020,2,8,Saturday
2,200320258,1700,3,Southwest,356,1,480,BIKE - STOLEN,0344 1251,19,...,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002,2020,11,4,Wednesday
3,200907217,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,19,...,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387,2020,3,10,Tuesday
4,220614831,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,1822 1501 0930 2004,28,...,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277,2020,8,17,Monday


In [66]:
print(df.dtypes)

DR_NO               int64
TIME OCC            int64
AREA                int64
AREA NAME          object
Rpt Dist No         int64
Part 1-2            int64
Crm Cd              int64
Crm Cd Desc        object
Mocodes            object
Vict Age            int64
Vict Sex           object
Vict Descent       object
Premis Cd         float64
Premis Desc        object
Weapon Used Cd    float64
Weapon Desc        object
Status             object
Status Desc        object
Crm Cd 1          float64
Crm Cd 2          float64
Crm Cd 3          float64
Crm Cd 4          float64
LOCATION           object
Cross Street       object
LAT               float64
LON               float64
Year OCC            int32
Month OCC           int32
DAY OCC             int32
DAY WEEK OCC       object
dtype: object


In [67]:
# Group all the assualts
df.loc[df['Crm Cd Desc'].str.contains('ASSAULT'), 'Crm Cd Desc'] = 'ASSAULT'

# CONVERT VEHICLE RELATED ISSUES TO VEHICLE ALTERCATION
df.loc[df['Crm Cd Desc'].str.contains('VEHICLE'), 'Crm Cd Desc'] = 'VEHICLE ALTERCATION'

# Group all the burglary
df.loc[df['Crm Cd Desc'].str.contains('BURGLARY'), 'Crm Cd Desc'] = 'BURGLARY'

# CONVERT THEFT OF IDENTITY TO IMPERSONIFICATION
df.loc[df['Crm Cd Desc'].str.contains('THEFT OF IDENTITY'), 'Crm Cd Desc'] = 'IMPERSONIFICATION'

# GROUP ALL THEFTS
df.loc[df['Crm Cd Desc'].str.contains('THEFT'), 'Crm Cd Desc'] = 'THEFT'
# df.loc[df['Crm Cd Desc'].str.contains('ROBBERY'), 'Crm Cd Desc'] = 'THEFT'
df.loc[df['Crm Cd Desc'].str.contains('STOLEN'), 'Crm Cd Desc'] = 'THEFT'

df.loc[df['Crm Cd Desc'].str.contains('RAPE'), 'Crm Cd Desc'] = 'SEXUAL HARASSMENT'
df.loc[df['Crm Cd Desc'].str.contains('SEXUAL'), 'Crm Cd Desc'] = 'SEXUAL HARASSMENT'

df.loc[df['Crm Cd Desc'].str.contains('VANDALISM'), 'Crm Cd Desc'] = 'VANDALISM'


In [68]:
# Filter out crimes with counts less than 1000
crime_counts = df['Crm Cd Desc'].value_counts()

crimes_to_keep = crime_counts[crime_counts >= 2000].index
df = df[df['Crm Cd Desc'].isin(crimes_to_keep)]

# Print the filtered DataFrame
df

,DR_NO,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,...,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Year OCC,Month OCC,DAY OCC,DAY WEEK OCC
0,190326475,2130,7,Wilshire,784,1,510,VEHICLE ALTERCATION,NaN,0,...,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,2020,3,1,Sunday
1,200106753,1800,1,Central,182,1,330,VEHICLE ALTERCATION,1822 1402 0344,47,...,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628,2020,2,8,Saturday
2,200320258,1700,3,Southwest,356,1,480,THEFT,0344 1251,19,...,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002,2020,11,4,Wednesday
3,200907217,2037,9,Van Nuys,964,1,343,THEFT,0325 1501,19,...,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387,2020,3,10,Tuesday
4,220614831,1200,6,Hollywood,666,2,354,IMPERSONIFICATION,1822 1501 0930 2004,28,...,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277,2020,8,17,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932135,241605270,1920,16,Foothill,1681,2,888,TRESPASSING,0913 1202 1501,79,...,NaN,NaN,8700 SHARP AV,NaN,34.2278,-118.4051,2024,2,11,Sunday
932136,241604405,720,16,Foothill,1673,1,230,ASSAULT,0400 1822 1310 0416,36,...,NaN,NaN,8700 LANKERSHIM BL,NaN,34.2271,-118.3878,2024,1,13,Saturday
932137,242106032,2330,21,Topanga,2103,1,420,VEHICLE ALTERCATION,385,44,...,NaN,NaN,22700 NAPA ST,NaN,34.2260,-118.6214,2024,2,17,Saturday
932138,242004546,1510,20,Olympic,2013,2,624,ASSAULT,2047 0400 0429 0444 1202 1822,80,...,NaN,NaN,HOBART BL,3RD ST,34.0690,-118.3054,2024,1,16,Tuesday


In [69]:
#removes duplicate rows according to "event_unique_id"
print('Original Data Size after dropping Duplicates')
df = df.drop_duplicates(subset='DR_NO',keep='first')
df.shape

Original Data Size after dropping Duplicates


(905481, 30)

In [70]:
drop_colmns = ['DR_NO','DAY OCC' ,'AREA','Premis Desc', 'Rpt Dist No','Part 1-2','Mocodes','Weapon Used Cd','Premis Cd','Crm Cd 1','Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4','Status Desc', 'Cross Street']
df_dropped = df.drop(columns=drop_colmns)
df_dropped.head()

,TIME OCC,AREA NAME,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Weapon Desc,Status,LOCATION,LAT,LON,Year OCC,Month OCC,DAY WEEK OCC
0,2130,Wilshire,510,VEHICLE ALTERCATION,0,M,O,NaN,AA,1900 S LONGWOOD AV,34.0375,-118.3506,2020,3,Sunday
1,1800,Central,330,VEHICLE ALTERCATION,47,M,O,NaN,IC,1000 S FLOWER ST,34.0444,-118.2628,2020,2,Saturday
2,1700,Southwest,480,THEFT,19,X,X,NaN,IC,1400 W 37TH ST,34.0210,-118.3002,2020,11,Wednesday
3,2037,Van Nuys,343,THEFT,19,M,O,NaN,IC,14000 RIVERSIDE DR,34.1576,-118.4387,2020,3,Tuesday
4,1200,Hollywood,354,IMPERSONIFICATION,28,M,H,NaN,IC,1900 TRANSIENT,34.0944,-118.3277,2020,8,Monday


In [71]:
df = df[df['Crm Cd Desc'] != 'VIOLATION OF COURT ORDER']
# df = df[df['Crm Cd Desc'] != 'SEX,UNLAWFUL(INC MUTUAL CONSENT, PENETRATION W/ FRGN OBJ']
df = df[df['Crm Cd Desc'] != 'LETTERS, LEWD  -  TELEPHONE CALLS, LEWD']
df = df[df['Crm Cd Desc'] != 'VIOLATION OF RESTRAINING ORDER ']
# df = df[df['Crm Cd Desc'] != 'CONTEMPT OF COURT ']

df

,DR_NO,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,...,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Year OCC,Month OCC,DAY OCC,DAY WEEK OCC
0,190326475,2130,7,Wilshire,784,1,510,VEHICLE ALTERCATION,NaN,0,...,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,2020,3,1,Sunday
1,200106753,1800,1,Central,182,1,330,VEHICLE ALTERCATION,1822 1402 0344,47,...,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628,2020,2,8,Saturday
2,200320258,1700,3,Southwest,356,1,480,THEFT,0344 1251,19,...,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002,2020,11,4,Wednesday
3,200907217,2037,9,Van Nuys,964,1,343,THEFT,0325 1501,19,...,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387,2020,3,10,Tuesday
4,220614831,1200,6,Hollywood,666,2,354,IMPERSONIFICATION,1822 1501 0930 2004,28,...,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277,2020,8,17,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932135,241605270,1920,16,Foothill,1681,2,888,TRESPASSING,0913 1202 1501,79,...,NaN,NaN,8700 SHARP AV,NaN,34.2278,-118.4051,2024,2,11,Sunday
932136,241604405,720,16,Foothill,1673,1,230,ASSAULT,0400 1822 1310 0416,36,...,NaN,NaN,8700 LANKERSHIM BL,NaN,34.2271,-118.3878,2024,1,13,Saturday
932137,242106032,2330,21,Topanga,2103,1,420,VEHICLE ALTERCATION,385,44,...,NaN,NaN,22700 NAPA ST,NaN,34.2260,-118.6214,2024,2,17,Saturday
932138,242004546,1510,20,Olympic,2013,2,624,ASSAULT,2047 0400 0429 0444 1202 1822,80,...,NaN,NaN,HOBART BL,3RD ST,34.0690,-118.3054,2024,1,16,Tuesday


In [72]:
df['Crm Cd Desc'].value_counts()

Crm Cd Desc
VEHICLE ALTERCATION                       235766
ASSAULT                                   194503
THEFT                                     136399
VANDALISM                                  80589
BURGLARY                                   60910
IMPERSONIFICATION                          57539
ROBBERY                                    31683
CRIMINAL THREATS - NO WEAPON DISPLAYED     18975
TRESPASSING                                14495
BRANDISH WEAPON                            14362
VIOLATION OF RESTRAINING ORDER             11626
SEXUAL HARASSMENT                           9885
OTHER MISCELLANEOUS CRIME                   6689
ATTEMPTED ROBBERY                           4770
PICKPOCKET                                  2784
DISCHARGE FIREARMS/SHOTS FIRED              2691
CONTEMPT OF COURT                           2686
ARSON                                       2515
BATTERY POLICE (SIMPLE)                     2507
Name: count, dtype: int64

In [73]:
df.shape

(891374, 30)

In [74]:
df_grouped = df_dropped.groupby(df_dropped['Year OCC'])

In [75]:
#Analysis by year
df_2020 = df_grouped.get_group(2020)
df_2021 = df_grouped.get_group(2021)
df_2022 = df_grouped.get_group(2022)
df_2023 = df_grouped.get_group(2023)
df_2024 = df_grouped.get_group(2024)

In [76]:
df_2020_grouped = df_2020.groupby(df_2020['Crm Cd Desc']).count()
# df_2021_grouped = df_2021.groupby(df_2021['Crm Cd Desc']).count()
# df_2022_grouped = df_2022.groupby(df_2022['Year OCC']).count()
# df_2023_grouped = df_2023.groupby(df_2023['Year OCC']).count()
# df_2024_grouped = df_2024.groupby(df_2024['Year OCC']).count()
# df_2020_grouped

In [77]:
plot = df_2020_grouped.iloc[:,0]
plot = pd.DataFrame(plot)
plot.columns = ['Number of Cases']
ax = plot.plot(kind='barh',figsize=(100,100),title='Number of Crimes in LA 2015')

In [78]:
df.head()

,DR_NO,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,...,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Year OCC,Month OCC,DAY OCC,DAY WEEK OCC
0,190326475,2130,7,Wilshire,784,1,510,VEHICLE ALTERCATION,NaN,0,...,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,2020,3,1,Sunday
1,200106753,1800,1,Central,182,1,330,VEHICLE ALTERCATION,1822 1402 0344,47,...,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628,2020,2,8,Saturday
2,200320258,1700,3,Southwest,356,1,480,THEFT,0344 1251,19,...,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002,2020,11,4,Wednesday
3,200907217,2037,9,Van Nuys,964,1,343,THEFT,0325 1501,19,...,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387,2020,3,10,Tuesday
4,220614831,1200,6,Hollywood,666,2,354,IMPERSONIFICATION,1822 1501 0930 2004,28,...,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277,2020,8,17,Monday


#### Convert to numerical data using factorization

In [101]:

col_list = ['Year OCC', 'Month OCC', 'DAY WEEK OCC', 'TIME OCC', 'Crm Cd Desc', 'AREA NAME', 'LOCATION','Premis Desc']


df2 = df[col_list]
df2 = df2[df2['Year OCC'] > 2021]

#Factorize dependent variable column:
crime_var = pd.factorize(df2['Crm Cd Desc'])
df2['Crm Cd Desc'] = crime_var[0]
definition_list_MCI = crime_var[1]

#factorize independent variables:
premise_var = pd.factorize(df2['Premis Desc'])
df2['Premis Desc'] = premise_var[0]
definition_list_premise = premise_var[1]

#factorize occurenceyear:
year_var = pd.factorize(df2['Year OCC'])
df2['Year OCC'] = year_var[0]
definition_list_year = year_var[1]

#factorize occurencemonth:
month_var = pd.factorize(df2['Month OCC'])
df2['Month OCC'] = month_var[0]
definition_list_month = month_var[1]

#factorize occurenceday:
# day_var = pd.factorize(df2['DAY OCC'])
# df2['DAY OCC'] = day_var[0]
# definition_list_day = day_var[1]

#factorize occurencedayofweek:
dayweek_var = pd.factorize(df2['DAY WEEK OCC'])
df2['DAY WEEK OCC'] = dayweek_var[0]
definition_list_day = dayweek_var[1]

#factorize division:
division_var = pd.factorize(df2['AREA NAME'])
df2['AREA NAME'] = division_var[0]
definition_list_division = division_var[1]

#factorize HOOD_ID:
hood_var = pd.factorize(df2['LOCATION'])
df2['LOCATION'] = hood_var[0]
definition_list_hood = hood_var[1]

#factorize occurencehour:
hour_var = pd.factorize(df2['TIME OCC'])
df2['TIME OCC'] = hour_var[0]
definition_list_hour = hour_var[1]


In [102]:
df2

,Year OCC,Month OCC,DAY WEEK OCC,TIME OCC,Crm Cd Desc,AREA NAME,LOCATION,Premis Desc
112354,0,0,0,0,0,0,0,0
358622,1,1,1,1,1,1,1,1
359154,1,2,0,2,1,2,2,1
359543,1,3,2,3,1,3,3,1
360264,1,4,3,3,1,4,4,1
...,...,...,...,...,...,...,...,...
932135,0,10,4,327,2,20,26451,0
932136,0,0,3,476,3,20,1468,1
932137,0,10,3,58,1,18,48593,20
932138,0,0,5,412,3,2,7148,70


#### Feature Scaling

In [ ]:
# Extract the feature columns from df2
X = df2.drop(columns=['Crm Cd Desc'])

# Initialize StandardScaler
scaler = StandardScaler()

# Scale the feature columns
X_scaled = scaler.fit_transform(X)

# Replace the original feature columns with the scaled ones
df2[X.columns] = X_scaled

In [ ]:
# Extract the feature columns for regular features
x = df2[['AREA NAME', 'Premis Desc', 'DAY WEEK OCC']].values
y = df2['Crm Cd Desc'].values

# Split the data for regular features
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=21)

# Initialize OneHotEncoder
binary_encoder = OneHotEncoder(categories='auto')

# Encode the features for one-hot encoding
encoded_X = binary_encoder.fit_transform(x)

# Split the encoded features for one-hot encoding
X_train_OH, X_test_OH, y_train_OH, y_test_OH = train_test_split(encoded_X, y, test_size=0.25, random_state=21)

#### Random Forest

In [106]:

classifier = RandomForestClassifier(n_estimators = 100 , criterion = 'entropy', random_state = 42)

classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
unique_classes = np.unique(np.concatenate((y_test, y_pred)))
target_names = [definition_list_MCI[i] for i in unique_classes]

print("Accuracy of Random Forest : ",accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=target_names))

Accuracy of Random Forest :  0.5053925798101812
[[ 3305   484     7  2381  1901    39    83   376     0     8     1     0
     12     0     0     0     0    12     1]
 [  477 29686     4  1419   946     8   363   100     0     2     1     0
      0     1     0     0     0    32     0]
 [  330   118   603   585   383     2    29   136     1     5     0     0
      4     0     0     2     1     2     0]
 [ 1701  7349   242 13885  2347    91   265   422     8    13     0     4
     12     2     0     1     2   110     2]
 [ 1609  2179    95  4393 10859   147   208   471     2     7     0     4
     10     1     0     3     1   201     1]
 [  199  1629    47  1250   954   140    27    40     0     0     0     0
      3     0     1     0     0     4     0]
 [  711  2530    56  2221  1398    24  2961   167    11     5     0     1
      5     4     0     5     0    12     2]
 [ 1411   226     0  5377   723     9    17  1531     0     3     0     0
      0     0     0     0     0     2     0]


#### Decision Tree

In [107]:
classifier = DecisionTreeClassifier(random_state=42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
unique_classes = np.unique(np.concatenate((y_test, y_pred)))
target_names = [definition_list_MCI[i] for i in unique_classes]

print("Accuracy of Decision Tree : ", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=target_names))

Accuracy of Decision Tree :  0.5054564918671908
[[ 3393   477     7  2355  1884    24    69   374     0     6     1     0
      9     0     0     0     0    11     0]
 [  503 29703     4  1431   917     5   348    98     0     1     1     0
      0     0     0     0     0    28     0]
 [  340   128   605   583   377     2    20   136     1     2     0     0
      2     0     0     2     1     2     0]
 [ 1771  7354   250 13871  2339    69   247   419     8     9     0     4
      5     2     0     1     2   104     1]
 [ 1697  2215    96  4395 10837   105   174   465     2     4     0     2
      5     1     0     3     1   189     0]
 [  213  1632    47  1254   976   110    18    39     0     0     0     0
      1     0     0     0     0     4     0]
 [  758  2550    57  2208  1381    19  2942   164    11     3     0     1
      1     3     0     4     0    11     0]
 [ 1418   228     1  5373   726     4    17  1530     0     1     0     0
      0     0     0     0     0     1     0]


c:\Users\TOBENNA\Desktop\FINAL YEAR PROJECT\CRIME IN LA\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\TOBENNA\Desktop\FINAL YEAR PROJECT\CRIME IN LA\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\TOBENNA\Desktop\FINAL YEAR PROJECT\CRIME IN LA\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

#### SVM

In [108]:
classifier = SVC(kernel='rbf', random_state=42)  # You can choose different kernels here
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
unique_classes = np.unique(np.concatenate((y_test, y_pred)))
target_names = [definition_list_MCI[i] for i in unique_classes]

print("Accuracy of SVM : ", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
classifier = SVC(kernel='sigmoid', random_state=42)  # You can choose different kernels here
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
unique_classes = np.unique(np.concatenate((y_test, y_pred)))
target_names = [definition_list_MCI[i] for i in unique_classes]

print("Accuracy of SVM : ", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=target_names))

In [85]:
# y_pred = classifier.predict(X_test)

In [86]:
# y_pred = classifier.predict(np.array([1200, 2,]).reshape(1, -1))
# print(y_pred)

In [87]:
# print(X_test)

In [88]:
# actual_crime_labels = []

# # Loop through each actual crime code and map it to the corresponding label
# for actual_crime_code in y_test:
#     actual_crime_label = definition_list_MCI[actual_crime_code]
#     actual_crime_labels.append(actual_crime_label)

# # Create a list to store the predicted crime labels
# predicted_crime_labels = []

# # Loop through each predicted crime code and map it to the corresponding label
# for predicted_crime_code in y_pred:
#     predicted_crime_label = definition_list_MCI[predicted_crime_code]
#     predicted_crime_labels.append(predicted_crime_label)

# # Create a list to store the original labels of the time
# original_time_labels = []

# # Loop through each time value in X_test and map it to the corresponding label
# for time_value in X_test[:, 0]:
#     original_time_label = definition_list_hour[time_value]
#     original_time_labels.append(original_time_label)

# # Create a list to store the original labels of the area
# original_area_labels = []

# # Loop through each area value in X_test and map it to the corresponding label
# for area_value in X_test[:, 1]:
#     original_area_label = definition_list_division[area_value]
#     original_area_labels.append(original_area_label)

# # Create a list to store the original labels of the day
# original_day_labels = []

# # Loop through each day value in X_test and map it to the corresponding label
# for day_value in X_test[:, 2]:
#     original_day_label = definition_list_day[day_value]
#     original_day_labels.append(original_day_label)

# # Create the testing_results DataFrame with 'Time', 'Actual Crime', 'Predicted Crime', 'Predicted Crime Label', and 'Original Time Label' columns
# testing_results = pd.DataFrame({'Time': original_time_labels,  # Use original time labels
#                                 'Area': original_area_labels,
#                                 'Week Day': original_day_labels,
#                                 'Actual Crime Label': actual_crime_labels,
#                                 'Predicted Crime Label': predicted_crime_labels})  # Add the predicted crime labels

# # Display the testing results DataFrame
# testing_results

In [89]:
# predicted_crime = classifier.predict(np.array(500).reshape(1, -1))

# print('predicted_crime: ', predicted_crime)

# predicted_crime_label = definition_list_MCI[predicted_crime]
# print(predicted_crime_label)

In [90]:
# classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
# classifier.fit(X_train_OH, y_train_OH)
# y_pred_OH = classifier.predict(X_test_OH)

# print("Accuracy of Random Forest with OneHotEncoder : ",accuracy_score(y_test, y_pred))
# print(confusion_matrix(y_test_OH, y_pred_OH))
# print(classification_report(y_test_OH,y_pred_OH, target_names=definition_list_MCI))